<a href="https://colab.research.google.com/github/engbJapan/Programming/blob/main/Python/Issue/reconstruct_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#構文解析の拡張がBNF(EBNF）定義で可能だったんか。

Lark!!

https://lark-parser.readthedocs.io/en/latest/examples/index.html

https://github.com/lark-parser/lark


REF

https://developers.10antz.co.jp/archives/2007

https://zenn.dev/ookuwab/articles/9137f640a88bd8#%E5%8F%82%E8%80%83%E3%81%AB%E3%81%97%E3%81%9F%E3%82%B5%E3%82%A4%E3%83%88

https://blog.panicblanket.com/archives/6279

https://qiita.com/k5trismegistus/items/358d58bcd50eb3f67fe8


PythonGrammer

https://docs.python.org/ja/3.7/reference/grammar.html


In [4]:
%matplotlib inline


# Reconstruct Python

Demonstrates how Lark's experimental text-reconstruction feature can recreate
functional Python code from its parse-tree, using just the correct grammar and
a small formatter.


In [5]:
%pip install lark -qq
from lark import Lark

#ここらをモディファイ
grammar = r"""
start: list | dict

list: "[" _seperated{atom, ","} "]"
dict: "{" _seperated{key_value, ","} "}"
key_value: atom ":" atom

_seperated{x, sep}: x (sep x)*  // Define a sequence of 'x sep x sep x ...'

atom: NUMBER | ESCAPED_STRING

%import common (NUMBER, ESCAPED_STRING, WS)
%ignore WS
"""


parser = Lark(grammar)

#ここも、構文木を見やすくするためParseオブジェクトにprettyメソッドを追加して出すようにモディファイ
print(parser.parse('[1, "a", 2]').pretty())
print(parser.parse('{"a": 2, "b": 6}').pretty())

start
  list
    atom	1
    atom	"a"
    atom	2

start
  dict
    key_value
      atom	"a"
      atom	2
    key_value
      atom	"b"
      atom	6



## Ref1
https://developers.10antz.co.jp/archives/2007

functionの日本語化の実行例
（拡張ではなく、この宣言内のみの対応になっとる）

そして実行し、動く

In [13]:
%pip install lark -qq
from lark import Lark, Transformer
 
def out(string):
    print(string[0])
 
class Main(Transformer):
    def __init__(self):
        self._functions = {}
 
    def function(self, token):
        self._functions[token[0]] = token[1:]
    
    def function_call(self, token):
        function = self._functions[token[0]]
        for state in function:
            eval(state.data)(state.children)
 
    def code_block(self, tree):
        return tree[0]
 
    def new_symbol(self, token):
        return token[0].value
    
    def symbol(self, token):
        return token[0].value
 
    def string(self, token):
        return token[0][1:-1]
 
 
# グラマー定義（EBNF記法での）を変数に紐づけ
# function キーワードを関数として、Cライクなブロック記法を導入した物
# 参照元の手法は外部ファイルとして居るところをダイレクトに変数に三重引用符書き綴り紐づけた
grammar = '''
?start: statement+
 
?statement: function
    | instruction
    | function_call
 
code_block: "{" statement+ "}"
function: "関数" new_symbol "(" ")" code_block
function_call: symbol "(" ")"
instruction: "出力" "(" string ")" -> out
 
string    : ESCAPED_STRING
symbol    : WORD
new_symbol: WORD
 
%import common.ESCAPED_STRING
 
%import common.WORD
 
%import common.WS
 
%ignore WS
'''
# 参照元の手法は外部ファイルとして居るところをダイレクトに変数にfリテラルで紐づけた
#ので以下のファイルからのリードやらの面倒な処理削除
#with open('./grammar.lark', 'r', encoding='utf-8') as a_file:
#    grammar = ''.join([line for line in a_file])
 
parser = Lark(grammar, parser='lalr', transformer=Main())
# 参照元の実行コードをこれまた、三重引用符で書き綴り与えた
parser.parse('''
関数 main(){
        出力("ようこそ❢")
}
main()
'''
)

ようこそ❢


Tree(Token('RULE', 'start'), [None, None])

In [12]:

from lark import Token, Lark
from lark.reconstruct import Reconstructor
from lark.indenter import PythonIndenter

# Official Python grammar by Lark
python_parser3 = Lark.open_from_package('lark', 'python.lark', ['grammars'],
                                        parser='lalr', postlex=PythonIndenter(), start='file_input',
                                        maybe_placeholders=False    # Necessary for reconstructor
                                        )

SPACE_AFTER = set(',+-*/~@<>="|:')
SPACE_BEFORE = (SPACE_AFTER - set(',:')) | set('\'')


def special(sym):
    return Token('SPECIAL', sym.name)

def postproc(items):
    stack = ['\n']
    actions = []
    last_was_whitespace = True
    for item in items:
        if isinstance(item, Token) and item.type == 'SPECIAL':
            actions.append(item.value)
        else:
            if actions:
                assert actions[0] == '_NEWLINE' and '_NEWLINE' not in actions[1:], actions

                for a in actions[1:]:
                    if a == '_INDENT':
                        stack.append(stack[-1] + ' ' * 4)
                    else:
                        assert a == '_DEDENT'
                        stack.pop()
                actions.clear()
                yield stack[-1]
                last_was_whitespace = True
            if not last_was_whitespace:
                if item[0] in SPACE_BEFORE:
                    yield ' '
            yield item
            last_was_whitespace = item[-1].isspace()
            if not last_was_whitespace:
                if item[-1] in SPACE_AFTER:
                    yield ' '
                    last_was_whitespace = True
    yield "\n"


class PythonReconstructor:
    def __init__(self, parser):
        self._recons = Reconstructor(parser, {'_NEWLINE': special, '_DEDENT': special, '_INDENT': special})

    def reconstruct(self, tree):
        return self._recons.reconstruct(tree, postproc)


def test():
    python_reconstructor = PythonReconstructor(python_parser3)

#    self_contents = open(__file__).read()

    self_contents = f'''statement: function| instruction| function_call
code_block: "{" statement+ "}"
function: "関数" new_symbol "(" ")" code_block
function_call: symbol "(" ")"
instruction: "出力" "(" string ")" -> out
 
string    : ESCAPED_STRING
symbol    : WORD
new_symbol: WORD
 
%import common.ESCAPED_STRING
 
%import common.WORD
 
%import common.WS
 
%ignore WS'''
    #    tree = python_parser3.parse(self_contents+'\n')
    tree = python_parser3.parse(self_contents+'\n')
    output = python_reconstructor.reconstruct(tree)

    tree_new = python_parser3.parse(output)
    print(tree.pretty())
    print(tree_new.pretty())
    # assert tree.pretty() == tree_new.pretty()
    assert tree == tree_new

    print(output)


if __name__ == '__main__':
    test()

UnexpectedToken: ignored